In [1]:
#GETTING MY ID:
import json

id = open('myID1.json', 'r')
mi = json.load(id)

from azureml.core import Workspace

ws = Workspace.create(name="Bankruptancy_pred2",
                      subscription_id = mi["my_id"],
                      resource_group = "model1.2",
                      location = "centralindia")

KeyboardInterrupt: 

In [ ]:
from azureml.core.model import Model

registered_model = Model.register(model_path="bankruptancy_pred.pkl",
                                  model_name="bankruptancy_pred_model",
                                  workspace=ws)

In [ ]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-bankruptancy")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=['pandas','scikit-learn'])

In [ ]:
################ score.py ###################

scorepy = """
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def init():
  global model
  model_path = Model.get_model_path('bankruptancy_pred_model')
  model = joblib.load(model_path)

def run(raw_data):
  try:
    data = json.loads(raw_data)['data']
    data = pd.DataFrame(data)
    data_dummies = data([[' ROA(C) before interest and depreciation before interest', ' ROA(A) before interest and % after tax', ' ROA(B) before interest and depreciation after tax', ' Net Value Per Share (B)', ' Net Value Per Share (A)', ' Net Value Per Share (C)', ' Persistent EPS in the Last Four Seasons', ' Operating Profit Per Share (Yuan ¥)', ' Per Share Net profit before tax (Yuan ¥)', ' Debt ratio %', ' Net worth/Assets', ' Borrowing dependency', ' Operating profit/Paid-in capital', ' Net profit before tax/Paid-in capital', ' Working Capital to Total Assets', ' Current Liability to Assets', ' Working Capital/Equity', ' Current Liabilities/Equity', ' Retained Earnings to Total Assets', ' Total expense/Assets', ' Current Liability to Equity', ' Equity to Long-term Liability', ' Current Liability to Current Assets', ' Net Income to Total Assets', " Net Income to Stockholder's Equity", ' Liability to Equity']], axis=1)

    result = model.predict(data_dummies)

    return json.dumps({"result": result.tolist()})
  except Exception as e:
    return json.dumps({"error": str(e)})
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [ ]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=0.5, memory_gb=1) ## ASEGÚRENSE DE ASIGNAR SUFICIENTE MADERA PARA SUS MUEBLES.

service = Model.deploy(workspace=ws,
                       name='bankruptancy_service',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

In [ ]:
service.wait_for_deployment()

scoring_uri = service.scoring_uri

scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")
file.write(scoreuri)
file.close()